In [1]:
import joblib
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
test = pd.read_csv('my_data/1_my_test.csv')
# Определение признаков и целевой переменной
X_new = test.drop(columns=['variantid1', 'variantid2'])
# Предполагается, что X_new содержит новые данные для инференса
a = 1
# Загрузка обученных моделей
final_hgb = joblib.load('final_hgb_model.pkl')
# Получение предсказаний для каждой модели
hgb_preds = final_hgb.predict_proba(X_new)[:, 1]
# Усреднение предсказанийq
ensemble_preds = hgb_preds

submission = pd.DataFrame({
    'variantid1': test['variantid1'],
    'variantid2': test['variantid2'],
    'target': ensemble_preds
})

submission.to_csv('submission.csv', index=False)

In [3]:
dir = 'C:/Users/Николай/PycharmProjects/OZON E-CUP/competition/'

attributes = pd.read_parquet(f'{dir}attributes.parquet', engine='pyarrow')
attributes.set_index('variantid', inplace=True)

text = pd.read_parquet(f'{dir}text_and_bert.parquet', engine='pyarrow')
text.set_index('variantid', inplace=True)

In [4]:
indexes = []
for idx in range(len(ensemble_preds)):
    if ensemble_preds[idx] < 0.55 and ensemble_preds[idx] > 0.45:
        indexes.append(idx)

In [5]:
len(indexes)

796

In [6]:
a=0
for indx in indexes:
        
    v1 = test.at[indx, 'variantid1']
    v2 = test.at[indx, 'variantid2']
    cat1 = json.loads(attributes.loc[v1, 'categories'])['3']
    cat2 = json.loads(attributes.loc[v2, 'categories'])['3']
    if test.at[indx, 'm_cos'] != 1:
        continue
    print('cos', cosine_similarity([text.at[v1, 'name_bert_64']], [text.at[v2, 'name_bert_64']])[0,0])
    print('e_jac', test.at[indx, 'e_jac'])
    print('m_cos', test.at[indx, 'm_cos'])
    print('e_avg_cos', test.at[indx, 'e_avg_cos'])
    print('Индекс', indx)
    print('pred', ensemble_preds[indx])
    print('Назавние')
    print(text.at[v1, 'name'])
    print('\n')
    print(text.at[v2, 'name'])
    print(text.at[v1, 'name'] == text.at[v2, 'name'])
    print('------------------------------------------------------------------------------------------------------------')
    print('Описание')
    print(text.at[v1, 'description'])
    print('\n')
    print(text.at[v2, 'description'])
    print(text.at[v1, 'description'] == text.at[v2, 'description'])
    print('------------------------------------------------------------------------------------------------------------')
    print('Аттрибуты')
    dict1 = json.loads(attributes.at[v1, 'characteristic_attributes_mapping'])
    dict2 = json.loads(attributes.at[v2, 'characteristic_attributes_mapping'])
    # Преобразуем ключи словарей в множества
    print(dict1)
    print('\n')
    print(dict2)
    print('\n')
    print('\n')
    print('\n')
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    a+=1

cos 0.9973444730527213
e_jac 0.125
m_cos 1.0
e_avg_cos 0.5806642233413257
Индекс 1169
pred 0.5075419307764828
Назавние
Диффузор ароматический новогодний "Модерн", 50 мл, зелёный чай, "Богатство Аромата"


Диффузор ароматический новогодний Модерн , 50 мл, зелёный чай, Богатство Аромата , 1 набора
False
------------------------------------------------------------------------------------------------------------
Описание
None


Диффузор — это безопасный, экологически чистый и натуральный способ ароматизации помещения, является прекрасной альтернативой ароматизированным спреям. Чтобы аромат оставался насыщенным, 1–2 раза в день доставайте палочки и переворачивайте их, вставляя другим концом в стеклянную колбу. Интенсивность аромата зависит от влажности воздуха (в более влажном помещении летучесть аромата сильнее), проветриваемости помещения и количества палочек. 
False
------------------------------------------------------------------------------------------------------------
Аттрибуты
{'Ар